In [ ]:
import os
import imagehash
from PIL import Image

def remove_duplicate_images_phash(image_folder, output_folder, hash_threshold=5):
    """
    Perceptual Hashing을 사용하여 중복 이미지를 제거
    Args:
        image_folder (str): 원본 이미지 폴더 경로
        output_folder (str): 중복 제거 후 저장할 폴더 경로
        hash_threshold (int): 해시값 간 차이 허용 범위 (작을수록 엄격)
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    hash_dict = {}
    unique_images = []

    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        if not filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')):
            continue

        # Perceptual Hash 계산
        img = Image.open(image_path)
        img_hash = imagehash.phash(img)

        # 중복 체크
        is_duplicate = False
        for existing_hash in hash_dict:
            if abs(img_hash - existing_hash) <= hash_threshold:
                is_duplicate = True
                break
        
        if not is_duplicate:
            hash_dict[img_hash] = filename
            unique_images.append(filename)
            img.save(os.path.join(output_folder, filename))

    print(f"Unique images saved: {len(unique_images)}")
    return unique_images


In [5]:
import os
from PIL import Image
import imagehash
import shutil

def remove_duplicate_images_with_count(image_folder, json_folder, hash_difference=5):
    """
    중복 이미지를 제거하고, 제거 전후 이미지 개수를 출력합니다.
    
    Args:
        image_folder (str): 이미지가 저장된 폴더 경로.
        json_folder (str): 이미지와 매칭된 JSON 파일들이 있는 폴더 경로.
        hash_difference (int): 허용하는 해시 차이(민감도).
    """
    # 이미지 목록 가져오기
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Initial image count: {len(image_files)}")  # 초기 이미지 개수 출력

    # 해시값 저장을 위한 딕셔너리
    hash_dict = {}
    duplicates = []

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        # 이미지 해시 계산
        image = Image.open(image_path)
        image_hash = imagehash.average_hash(image)
        image.close()

        # 중복 여부 확인
        found_duplicate = False
        for stored_hash, stored_file in hash_dict.items():
            if abs(image_hash - stored_hash) <= hash_difference:
                duplicates.append(image_file)
                found_duplicate = True
                break
        
        if not found_duplicate:
            hash_dict[image_hash] = image_file  # 해시 딕셔너리에 추가

    # 중복 이미지 제거
    for duplicate_file in duplicates:
        # 이미지 삭제
        duplicate_image_path = os.path.join(image_folder, duplicate_file)
        os.remove(duplicate_image_path)

        # JSON 파일 삭제
        duplicate_json_path = os.path.join(json_folder, os.path.splitext(duplicate_file)[0] + ".json")
        if os.path.exists(duplicate_json_path):
            os.remove(duplicate_json_path)

    # 최종 이미지 개수 출력
    final_image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Final image count: {len(final_image_files)}")  # 중복 제거 후 이미지 개수 출력
    print(f"Removed {len(duplicates)} duplicate images.")  # 제거된 중복 이미지 개수 출력


In [13]:
# 경로 설정
image_folder = "/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/images"
json_folder = "/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/labels"

# 민감도 순차적으로 조정하며 실행
remove_duplicate_images_with_count(image_folder, json_folder, hash_difference=17)


Initial image count: 601
Final image count: 450
Removed 151 duplicate images.
